# A groundwater model for Schoonhoven

In this notebook we build a model for the area around Schoonhoven. We mainly
focus on the surface water. There are three water boards in the model area, of
which we download seasonal data about the stage of the surface water. For
locations without a stage from the water board, we deliniate information from a
Digital Terrain Model, to set a stage. Finally, for the river Lek, we build a
river package with a fixed stage of NAP+0.0 m.

## Import packages

In [ ]:
import os
import flopy
import matplotlib
import matplotlib.pyplot as plt
import nlmod
import numpy as np
import pandas as pd
import hydropandas as hpd
import geopandas as gpd
import rioxarray
from nlmod.dcs import DatasetCrossSection
from shapely.geometry import LineString, Point
import warnings

In [ ]:
print(f'nlmod version: {nlmod.__version__}')

nlmod.util.get_color_logger('INFO');

## Model settings
We define some model settings, like the name, the directory of the model files, the model extent and the time

In [ ]:
model_name = "Schoonhoven"
model_ws = "model9"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [116500, 120000, 439000, 442000]
time = pd.date_range("2015", "2022", freq="MS") # monthly timestep

## Download data
### AHN
Download the Digital Terrain model of the Netherlands (AHN) with a resolution of 0.5 meter.

In [ ]:
fname_ahn = os.path.join(cachedir, "ahn.tif")
if not os.path.isfile(fname_ahn):
    ahn = nlmod.read.ahn.get_ahn4(extent, identifier='AHN4_DTM_05m')
    ahn.rio.to_raster(fname_ahn)
ahn = rioxarray.open_rasterio(fname_ahn, mask_and_scale=True)

### layer 'waterdeel' from bgt
As the source of the location of the surface water bodies we use the 'waterdeel' layer of the Basisregistratie Grootschalige Topografie (BGT). This data consists of detailed polygons, maintained by dutch government agencies (water boards, municipalities and Rijkswatrstaat).

In [ ]:
bgt = nlmod.read.bgt.get_bgt(extent)

#### Add minimum surface height around surface water bodies
Get the minimum surface level in 1 meter around surface water levels and add these data to the column 'ahn_min'.

In [ ]:
if True:
    # use rasterstats
    from rasterstats import zonal_stats
    stats = zonal_stats(bgt.geometry.buffer(1.0), fname_ahn, stats="min")
    bgt["ahn_min"] = [x["min"] for x in stats]
else:
    # use geocube
    from geocube.api.core import make_geocube
    from functools import partial
    from geocube.rasterize import rasterize_image
    gc = make_geocube(
        vector_data=bgt.buffer(1.0).reset_index().rename_geometry('geometry'),
        measurements=["index"],
        like=ahn, # ensure the data are on the same grid
        rasterize_function=partial(rasterize_image, all_touched=True),
    )
    gc['ahn'] = ahn

    ahn_min = gc.groupby('index').min()['ahn'].to_pandas()
    ahn_min.index = ahn_min.index.astype(int)
    bgt['ahn_min'] = ahn_min

#### Plot 'bronhouder'
We can plot the column 'bronhouder' from the GeoDataFrame bgt. We see there are three water boards in this area (with codes starting with 'W').

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot("bronhouder", legend=True, ax=ax);

### level areas
For these three waterboards we download the level areas (peilgebieden): polygons with information about winter and summer stages.

In [ ]:
pg = nlmod.gwf.surface_water.download_level_areas(bgt, extent=extent)

#### Plot summer stage
The method download_level_areas() generates a dictionary with the name of the water boards as keys and GeoDataFrames as values. Each GeoDataFrame contains the columns summer_stage and winter_stage. Let's plot the summer stage, together with the location of the surface water bodies.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot(color="k", ax=ax)
for wb in pg:
    pg[wb].plot("summer_stage", ax=ax, vmin=-3, vmax=1, zorder=0)

#### Add stages to bgt-data
We then add the information from these level areas to the surface water bodies.

In [ ]:
bgt = nlmod.gwf.surface_water.add_stages_from_waterboards(bgt, pg=pg)

#### Plot summer stage of surface water bodies
We can plot the summer stage. There are some surface water bodies without a summer-stage, because the 'bronhouder' is not a water board. The main one is the river Lek, but there are also some surface water bodies without a summer stage more north.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
norm = matplotlib.colors.Normalize(vmin=-3, vmax=1)
cmap = "viridis"
bgt.plot("summer_stage", ax=ax, norm=norm, cmap=cmap)
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap)

If no information about the stage is available, a constant stage is set to the minimal height of the digital terrain model (AHN) near the surface water body. We can plot these values as well:

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot("ahn_min", ax=ax, norm=norm, cmap=cmap)
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap)

### REGIS
For the schematisation of the subsurface we use REGIS. Let's download this data for the required extent.

In [ ]:
regis = nlmod.read.get_regis(extent, cachedir=cachedir, cachename="regis.nc")
regis

We then create a regular grid, add nessecary variables (eg idomain) and fill nan's. For example, REGIS does not contain infomration about the hydraulic conductivity of the first layer ('HLc'). These NaN's are replaced by a default hydraulic conductivity (kh) of 1 m/d. This probably is not a good representation of the conductivity, but at least the model will run.

In [ ]:
ds = nlmod.to_model_ds(regis, model_name, model_ws, delr=100.0, delc=100.0)
ds

## Add grid refinement
With the refine method, we can add grid refinement. The model will then use the disv-package instead of the dis-package. We can also test if the disv-package gives the same results as the dis-package by not specifying refinement_features: ds = nlmod.grid.refine(ds).

This notebook can be run with or without running the cell below.

In [ ]:
refinement_features = [(bgt[bgt["bronhouder"] == "L0002"], 2)]
ds = nlmod.grid.refine(ds, refinement_features=refinement_features)

## Add information about time

In [ ]:
ds = nlmod.time.set_ds_time(ds, time=time)

## Add knmi recharge to the model dataset

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(
    ds, cachedir=cachedir, cachename="recharge.nc"
)
ds.update(knmi_ds)

## Create a groundwater flow model
Using the data from the xarray Dataset ds we generate a groundwater flow model.

In [ ]:
# create simulation 
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=0.0)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

# create storagee package
sto = nlmod.gwf.sto(ds, gwf)

## Process surface water
We cut the surface water bodies with the grid, set a default resistance of 1 day, and seperate the large river 'Lek' form the other surface water bodies.

In [ ]:
bed_resistance = 1.0

mg = nlmod.grid.modelgrid_from_ds(ds)
gi = flopy.utils.GridIntersect(mg, method="vertex")
bgt_grid = nlmod.grid.gdf_to_grid(bgt, ix=gi).set_index("cellid")

bgt_grid["cond"] = bgt_grid.area / bed_resistance
mask = bgt_grid["bronhouder"] == "L0002"
lek = bgt_grid[mask]
bgt_grid = bgt_grid[~mask]

### Lek as river
Model the river Lek as a river with a fixed stage of 0.5 m NAP

In [ ]:
lek["stage"] = 0.0
lek["rbot"] = -3.0
spd = nlmod.gwf.surface_water.build_spd(lek, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})

### Other surface water as drains
model the other surface water using the drain package, with a summer stage and a  winter stage

In [ ]:
drn = nlmod.gwf.surface_water.gdf_to_seasonal_pkg(bgt_grid, gwf, ds);

## Run the model

In [ ]:
nlmod.sim.write_and_run(sim, ds)

## Post-processing
### Get the simulated head

In [ ]:
head = nlmod.gwf.get_heads_da(ds)

### Plot the average head in the first layer on a map

In [ ]:
f, ax = nlmod.plot.get_map(extent)
norm = matplotlib.colors.Normalize(-2.5, 0.0)
pc = nlmod.plot.data_array(
    head.sel(layer="HLc").mean("time"), ds=ds, edgecolor="k", norm=norm
)
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none")

### Plot the average head in a cross-section, from north to south

In [ ]:
x = 118228.0
line = [(x, 439000), (x, 442000)]
f, ax = plt.subplots(figsize=(10, 6))
ax.grid()
dcs = DatasetCrossSection(ds, line, ax=ax, zmin=-100.0, zmax=10.0)
pc = dcs.plot_array(head.mean("time"), norm=norm, head=head.mean("time"))
# add labels with layer names
cbar = nlmod.plot.colorbar_inside(pc, bounds=[0.05, 0.05, 0.02, 0.9])
dcs.plot_grid()
dcs.plot_layers(alpha=0.0, min_label_area=1000)
f.tight_layout(pad=0.0)

### plot a time series at a certain location

In [ ]:
x = 118228
y = 439870
if ds.gridtype == "vertex":
    icelld2 = gi.intersect(Point(x, y))["cellids"][0]
    head_point = head[:, :, icelld2]
else:
    head_point = head.interp(x=x, y=y, method="nearest")
# only keep layers that are active at this location
head_point = head_point[:, ~head_point.isnull().all("time")]
head_point.plot.line(hue="layer", size=10);

### Compare with BRO measurements

### Plot some properties of the first layer
We can plot some properties of the first layer, called HLc. As REGIS does not contain data about hydraulic conductivities for this layer, default values of 1 m/d for kh and 0.1 m/d for hv are used, which can be seen in the graphs below.

In [ ]:
layer = "HLc"
f, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2)
variables = ["top", "kh", "botm", "kv"]
for i, variable in enumerate(variables):
    ax = axes.ravel()[i]
    if variable == "top":
        if layer == ds.layer[0]:
            da = ds["top"]
        else:
            da = ds["botm"][np.where(ds.layer == layer)[0][0] - 1]
    else:
        da = ds[variable].sel(layer=layer)
    pc = nlmod.plot.data_array(da, ds=ds, ax=ax)
    nlmod.plot.colorbar_inside(pc, ax=ax)
    ax.text(
        0.5,
        0.98,
        f"{variable} in layer {layer}",
        ha="center",
        va="top",
        transform=ax.transAxes,
    )

## Add pathlines

We create a modpath model which calculates the pathlines. We calculate the pathlines that start in the center of the modflow cells with a river boundary condition (the cells in the "Lek" river).

In [ ]:
# create a modpath model
mpf = nlmod.modpath.mpf(gwf)

# create the basic modpath package
_mpfbas = nlmod.modpath.bas(mpf)

# get the nodes from a package
nodes = nlmod.modpath.package_to_nodes(gwf, 'RIV_0', mpf)

# create a particle tracking group from cell centers
pg = nlmod.modpath.pg_from_pd(nodes, localx=0.5, localy=0.5, localz=0.5)

# create the modpath simulation file
mpsim = nlmod.modpath.sim(mpf, pg, 'forward', gwf=gwf)

In [ ]:
# run modpath model
nlmod.modpath.write_and_run(mpf, nb_path='10_modpath.ipynb')

In [ ]:
pdata = nlmod.modpath.load_pathline_data(mpf)

In [ ]:
def get_segments(x, y, segments=None):
    # split each flopath in multiple line segments
    return [np.column_stack([x[i:i+2], y[i:i+2]]) for i in range(len(x) - 1)]
    
def get_array(time, to_year=True):
    # for each line-segment use the average time as the color
    array = (time[:-1] + time[1:])/2
    if to_year:
        array = array / 365.25
    return array

cmap = plt.get_cmap('turbo')
norm = matplotlib.colors.BoundaryNorm([0, 1, 2, 5, 10, 25, 50, 100, 200, 500], cmap.N, extend='max')

# get line segments and color values
segments = []
array = []
for pid in np.unique(pdata['particleid']):
    pf = pdata[pdata['particleid']==pid]
    segments.extend(get_segments(pf['x'], pf['y']))
    array.extend(get_array(pf['time']))

f, ax = nlmod.plot.get_map(extent)
lc = matplotlib.collections.LineCollection(segments, cmap=cmap, norm=norm, array=array, linewidth=1.0)
line = ax.add_collection(lc)
nlmod.plot.colorbar_inside(line, label='Travel time (years)')

bgt.plot(ax=ax, edgecolor="k", facecolor="none");

In [ ]:
x = 118228.0
line = LineString([(x, 439000), (x, 442000)])

# get line segments and color values
segments=[]
array = []
for pid in np.unique(pdata["particleid"]):
    pf = pdata[pdata["particleid"] == pid]
    d = line.distance(Point(pf["x"][0], pf["y"][0]))
    if d < 200.0:
        x = [line.project(Point(x, y)) for x, y in zip(pf["x"], pf["y"])]
        segments.extend(get_segments(x, pf["z"]))
        array.extend(get_array(pf['time']))

f, ax = plt.subplots(figsize=(10, 6))
ax.grid()
dcs = DatasetCrossSection(ds, line, ax=ax, zmin=-100.0, zmax=10.0)
lc = matplotlib.collections.LineCollection(segments, cmap=cmap, norm=norm, array=array, linewidth=1.0)
line = ax.add_collection(lc)
nlmod.plot.colorbar_inside(line, label='Travel time (years)')
# add grid
dcs.plot_grid()
# add labels with layer names
dcs.plot_layers(alpha=0.0, min_label_area=1000)
f.tight_layout(pad=0.0)